In [1]:
%%bash
echo $PYTHONPATH

/home/marjanalbooyeh/projects/grits/:/home/marjanalbooyeh/projects/cmeutils/:/home/marjanalbooyeh/code/cme-lab/cmeutils/:/home/marjanalbooyeh/projects/polybinder/


In [2]:
from polybinder.system import System, Initializer
from polybinder.simulate import Simulation
import mbuild as mb
import hoomd
from mbuild.formats.hoomd_forcefield import to_hoomdsnapshot
import numpy as np

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
system_params = System(density=1.5,
                    molecule="PPS",
                    polymer_lengths=8,
                    n_compounds=500,
                    para_weight=1.0,
                   )

In [ ]:
%%time
PPS_rigid_bead = {'R': 'c1ccccc1'}
PPS_system = Initializer(system=system_params,
                        system_type='pack',
                        rigid_bead_smiles=PPS_rigid_bead)

generating compounds...
packing...
applying ff...


In [24]:
# pps_comp = PPS_system.mb_compounds[0]

In [25]:
# pps_comp_cg = CG_Compound(pps_comp, PPS_rigid_bead)
# bead_name = [list(pps_comp_cg.mapping.keys())[0].split('...')[0]]

In [26]:
# base_idx = np.array(list(map(np.array, list(pps_comp_cg.mapping.values())[0])))

In [27]:
# all_bead_idx = []
# for i in range(PPS_system.system_parms.n_compounds[0]):
#     all_bead_idx.extend((base_idx + (pps_comp.n_particles*i)).tolist())
    

In [28]:
# system_rigid_ids = np.full(len(PPS_system.system.atoms), -1)
# N_rigid_bodies = len(all_bead_idx)
# for i, idx in enumerate(all_bead_idx):
#     system_rigid_ids[list(idx)] = i


In [29]:
dt = 0.0001
tau_kT = 0.003
path = 'logs/test'
simulation = Simulation(system=PPS_system, dt=dt, tau_kt=tau_kT, logs_path=path)

Processing LJ and QQ
No charged groups found, ignoring electrostatics
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing periodic torsions


In [30]:
kT_init = 2.0
kT=1.5
shrink_n_steps = 1e6
quench_n_steps = 1e6

In [31]:
simulation.shrink(n_steps=shrink_n_steps, kT_init = kT_init, kT_final=kT)

In [32]:
simulation.quench(n_steps=quench_n_steps, kT=kT)

In [33]:
import hoomd
getattr(hoomd.md.nlist, "Cell")

hoomd.md.nlist.Cell

In [29]:
for force in simulation.integrator.forces:
    if np.any(force.torques):
        print(force.torques)

In [17]:
with simulation.sim.state.cpu_local_snapshot as local_snap:
    print(list(local_snap.particles.net_torque))

[HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([-35.43998778  95.87828659   5.58482124]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([0. 0. 0.]), HOOMDArray([  95.68573988 -111.80556817   66.61018717]), 

In [18]:
with simulation.sim.state.cpu_local_snapshot as local_snap:
    print(np.sum(local_snap.particles.net_torque))

597.5703482509191


In [54]:
simulation.sim.operations.integrator.rigid.body['R']

_HOOMDDict{'constituent_types': ['ca', 'ca', 'ca', 'ca', 'ca', 'ca'], 'positions': [(-0.19529553486151485, 0.010921415422309622, 0.32508352981420785), (0.16532738832622584, -0.09323183804033874, 0.3211156264734214), (0.35767697477272975, -0.10253227795710984, 3.142893735486041e-05), (0.19516972557362067, -0.01098399268148853, -0.3247863579869019), (-0.16528482830689484, 0.09324783312452212, -0.3213895072132047), (-0.35759372550414525, 0.10257886013211959, -5.4720024849075344e-05)], 'orientations': [(1.0, 0.0, 0.0, 0.0), (1.0, 0.0, 0.0, 0.0), (1.0, 0.0, 0.0, 0.0), (1.0, 0.0, 0.0, 0.0), (1.0, 0.0, 0.0, 0.0), (1.0, 0.0, 0.0, 0.0)], 'charges': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'diameters': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}